# Infill scenarios

Use the SSP projections for non-CO2, non-CH4 and non-N2O emissions.

Use CO2 as a lead variable. This might need a two-step infilling process:
1. split CO2 into FFI and AFOLU
2. use either total or FFI to infill other species

In [ ]:
import silicone.database_crunchers
from silicone.stats import rolling_window_find_quantiles
from silicone.utils import (
    _get_unit_of_variable,
    find_matching_scenarios,
    _make_interpolator,
    _make_wide_db,
    get_sr15_scenarios,
    download_or_load_sr15,
)